In [1]:
import json
import galois
from py_ecc.optimized_bn128 import FQ
from utils import GPoint, SRS, G1, G2, generator1, generator2, validate_point

import sha3

def numbers_to_hash(numbers, field) -> int:
    """Hash a number."""
    engine = sha3.keccak_256()
    for number in numbers:
        if isinstance(number, tuple):
            x, y, z = number
            engine.update(bytes(hex(int(x)), 'utf-8'))
            engine.update(bytes(hex(int(y)), 'utf-8'))
            engine.update(bytes(hex(int(z)), 'utf-8'))
        else:
            engine.update(bytes(hex(int(number)), 'utf-8'))
    return field(int(engine.hexdigest(), 16) % field.order)


G1 = generator1()
G2 = generator2()

def new_call(self, at, **kwargs):
    if isinstance(at, SRS):
        coeffs = self.coeffs[::-1]
        result = at.tau1[0] * coeffs[0]
        for i in range(1, len(coeffs)):
            result += at.tau1[i] * coeffs[i]
        return result

    return galois.Poly.original_call(self, at, **kwargs)

galois.Poly.original_call = galois.Poly.__call__
galois.Poly.__call__ = new_call

proof = json.load(open("proof.json", "r"))
p = proof.pop("Fp")

Fp = galois.GF(p)

circuit = json.load(open("circuit.json", "r"))
circuit.pop("Fp")
for k, v in circuit.items():
    if k in ["tau", "k1", "k2", "Fp", "omega", "n"]:
        circuit[k] = Fp(v)
    elif isinstance(v, bool):
        continue
    else:
        circuit[k] = galois.Poly(coeffs=v, field=Fp)
encrypted = circuit.pop("encrypted")
n = int(circuit["n"])
QM = circuit["QM"]
QL = circuit["QL"]
QR = circuit["QR"]
QO = circuit["QO"]
QC = circuit["QC"]
S1 = circuit["S1"]
S2 = circuit["S2"]
S3 = circuit["S3"]
Zh = circuit["Zh"]
L1 = circuit["L1"]
PI = circuit["PI"]
tau = circuit["tau"] if not encrypted else SRS(circuit["tau"], n)
k1 = circuit["k1"]
k2 = circuit["k2"]
omega = circuit["omega"]

for k, v in proof.items():
    if isinstance(v, list):
        proof[k] = [Fp(x) for x in v]
    elif isinstance(v, str):
        proof[k] = GPoint( *[FQ(int(x)) for x in v.strip('(').strip(')').split(',')])
    else:
        proof[k] = Fp(v)

round1 = [proof["A"], proof["B"], proof["C"]]
round2 = [proof["Z"]]
round3 = [proof["Tl"], proof["Tm"], proof["Th"]]
round4 = [proof["a_zeta"], proof["b_zeta"], proof["c_zeta"], proof["s1_zeta"], proof["s2_zeta"], proof["z_omega_zeta"]]
round5 = [proof["Wzeta"], proof["Womega_zeta"]]

# Verifier

In [2]:
# These evaluations are calculated beforehand during the setup phase
qm_exp = QM(tau)
ql_exp = QL(tau)
qr_exp = QR(tau)
qo_exp = QO(tau)
qc_exp = QC(tau)
s1_exp = S1(tau)
s2_exp = S2(tau)
s3_exp = S3(tau)

# Values provided by the prover (round 1 to 5) is a proof.
a_exp = round1[0]
b_exp = round1[1]
c_exp = round1[2]

z_exp = round2[0]

tl_exp = round3[0]
tm_exp = round3[1]
th_exp = round3[2]

# Note: verifier has to verify that the following values are in the correct Fp field
a_zeta, b_zeta, c_zeta, s1_zeta, s2_zeta, z_omega_zeta = round4

w_zeta_exp = round5[0]
w_omega_zeta_exp = round5[1]

# Note: verifier has to verify that the following values are on the curve
if encrypted:
    validate_point(qm_exp)
    validate_point(ql_exp)
    validate_point(qr_exp)
    validate_point(qo_exp)
    validate_point(qc_exp)
    validate_point(z_exp)
    validate_point(s1_exp)
    validate_point(s2_exp)
    validate_point(s3_exp)
    validate_point(tl_exp)
    validate_point(tm_exp)
    validate_point(th_exp)
    validate_point(a_exp)
    validate_point(b_exp)
    validate_point(c_exp)
    validate_point(w_zeta_exp)
    validate_point(w_omega_zeta_exp)

beta = numbers_to_hash(round1 + [0], Fp)
gamma = numbers_to_hash(round1 + [1], Fp)
alpha = numbers_to_hash(round1 + round2, Fp)
zeta = numbers_to_hash(round1 + round2 + round3, Fp)
v = numbers_to_hash(round1 + round2 + round3 + round4, Fp)
u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, Fp)
# u = Fp(2)

In [3]:
Zh_z = Zh(zeta)
L1_z = L1(zeta)
PI_z = PI(zeta)

r0 = (PI_z - L1_z * alpha**2 -
    (a_zeta + beta * s1_zeta + gamma) *
    (b_zeta + beta * s2_zeta + gamma) *
    (c_zeta + gamma) * z_omega_zeta * alpha)

In [4]:
D_exp = (qm_exp * a_zeta * b_zeta +
        ql_exp * a_zeta +
        qr_exp * b_zeta +
        qo_exp * c_zeta +
        qc_exp)

D_exp += (z_exp * (
        (a_zeta + beta * zeta + gamma) *
        (b_zeta + beta * zeta * k1 + gamma) *
        (c_zeta + beta * zeta * k2 + gamma) * alpha
        + L1_z * alpha**2 + u))

D_exp -= (s3_exp *
        (a_zeta + beta * s1_zeta + gamma) *
        (b_zeta + beta * s2_zeta + gamma) * 
        alpha * beta * z_omega_zeta)

D_exp -= ((tl_exp + 
        tm_exp * zeta**n  +
        th_exp * zeta**(2*n)) *
        Zh_z)

In [5]:
F_exp = (D_exp + 
        a_exp * v +
        b_exp * v**2 +
        c_exp * v**3 +
        s1_exp * v**4 +
        s2_exp * v**5)

print(f"F_exp = {F_exp}")

F_exp = (21657532057912584823646137870907508675278530212614641607595162151825597375368, 19543798916048237908748449407655822101386574292914020108748404837746222007343, 21445344355381160976137495432701574366375356479442092492047660418583174441572)


In [6]:
E_exp = (-r0 +
        v * a_zeta +
        v**2 * b_zeta +
        v**3 * c_zeta +
        v**4 * s1_zeta +
        v**5 * s2_zeta +
        u * z_omega_zeta)

if encrypted:
        E_exp = G1 * E_exp

print(f"E_exp = {E_exp}")

E_exp = (17459173416278814315753763562125347744266039940541381801726306915243484294997, 1307624642964067106841107479733080905212058915704695916522200128466010638763, 14305516036151272312492126809303825096099152061774973562069230877979496835992)


In [7]:
e1 = w_zeta_exp + w_omega_zeta_exp * u
e2 = (w_zeta_exp * zeta + w_omega_zeta_exp * (u * zeta * omega) +
    F_exp + (E_exp * Fp(p-1)))

if encrypted:
    pairing1 = tau.tau2.pair(e1)
    pairing2 = G2.pair(e2)

    print(f"pairing1 = {pairing1}")
    print(f"pairing2 = {pairing2}")

    assert pairing1 == pairing2, f"pairing1 != pairing2"
else:
    print("\n\n--- e1, e2 ---")
    print(f"e1 = {e1 * tau} = {e1} * tau")
    print(f"e2 = {e2}")
    assert e1 * tau == e2

pairing1 = (1995524244905634973051289338410047215939497810662085468048857028536236827485, 7387396249125994352231452489008895507424282268692284231480616357421276372009, 15980839134404896310461893125714008384730728107204167967568573327513965595322, 13403794397039826461380381384317254567755449884512264787211460655040373877204, 9682088073025662616013992059147658990785139150637334602832671584948870347999, 13878328290834720204712268808270529479160592520076597954814269036384099592167, 2868912482778317219476603830117389977599520723022394424834458120120508156279, 10362426014774102954381913156604555782316562211383026629166568025047376090021, 17980162332020806365668723229904072193728870974944251788323145130213624625951, 16559423327676627549564196964221226532959436263168667730480553316216437116883, 9755851071128066467510154810347311905641142318016715292805523665524304985451, 9547299747612497522533986006801129701873246236971751314135552554575950847582)
pairing2 = (1995524244905634973051289338410047

# Attack

In [12]:
a_zeta = Fp(1)

Zh_z = Zh(zeta)
L1_z = L1(zeta)
PI_z = PI(zeta)

r0 = (PI_z - L1_z * alpha**2 -
    (a_zeta + beta * s1_zeta + gamma) *
    (b_zeta + beta * s2_zeta + gamma) *
    (c_zeta + gamma) * z_omega_zeta * alpha)

D_exp = (qm_exp * a_zeta * b_zeta +
        ql_exp * a_zeta +
        qr_exp * b_zeta +
        qo_exp * c_zeta +
        qc_exp)

D_exp += (z_exp * (
        (a_zeta + beta * zeta + gamma) *
        (b_zeta + beta * zeta * k1 + gamma) *
        (c_zeta + beta * zeta * k2 + gamma) * alpha
        + L1_z * alpha**2 + u))

D_exp -= (s3_exp *
        (a_zeta + beta * s1_zeta + gamma) *
        (b_zeta + beta * s2_zeta + gamma) * 
        alpha * beta * z_omega_zeta)

D_exp -= ((tl_exp + 
        tm_exp * zeta**n  +
        th_exp * zeta**(2*n)) *
        Zh_z)

F_exp = (D_exp + 
        a_exp * v +
        b_exp * v**2 +
        c_exp * v**3 +
        s1_exp * v**4 +
        s2_exp * v**5)

E_exp = (-r0 +
        v * a_zeta +
        v**2 * b_zeta +
        v**3 * c_zeta +
        v**4 * s1_zeta +
        v**5 * s2_zeta +
        u * z_omega_zeta)

if encrypted:
        E_exp = G1 * E_exp


In [18]:
from sympy import symbols, Eq, solve

# Define the symbols
X, Y = symbols('X Y')

# Constants are assumed to be known and are represented as symbols for generality
A, B, C, _u, z, w = symbols('A B C u z w')

# Define the equations based on the provided system
eq1 = Eq(X + _u*Y, A)
eq2 = Eq(z * X + _u*z*w*Y + C, B)

# Solve the system of equations
solutions = solve((eq1, eq2), (X, Y))
solutions

_A = e1
_B = e2
_C = F_exp - E_exp

y = (_B - _C - _A * zeta) * (u * omega * zeta - u * zeta)**(-1)

print(f"y = {y}")
print(f"w_omega_zeta_exp = {w_omega_zeta_exp}")

x = (_A* omega *zeta - _B + _C) * (omega * zeta - zeta)**(-1)

print(f"x = {x}")
print(f"w_zeta_exp = {w_zeta_exp}")

e1 = x + y * u
e2 = (x * zeta + y * (u * zeta * omega) +
    F_exp + (E_exp * Fp(p-1)))
if encrypted:
    pairing1 = tau.tau2.pair(e1)
    pairing2 = G2.pair(e2)
    
    print(f"pairing1 = {pairing1}")
    print(f"pairing2 = {pairing2}")

    assert pairing1 == pairing2, f"pairing1 != pairing2"
else:
    assert e1 * tau == e2

y = (3340633048515885749195077327103191171848939852705804118240909137266413070755, 8004942409414800878798507027817149490072951166443071226165112740241220588653, 5121099607657564547393014277939623727825204631778375750651619082201251665464)
w_omega_zeta_exp = (14836619538049180941356686805788535341236789774424797341006313436988938820722, 11787884999407183532145357481761688921577099468081343952825562512756228903485, 20964417647469560222954881941929895863002336481485673611680588748743868043406)
x = (15591120245871154778233023267816938219166778112307998500232599114619589012803, 13894799655458462368911110961929904249336202913543609870695305551089689977769, 11768220359607897269868772060606222732527171299143814755558808334882527438126)
w_zeta_exp = (424404092856860038150039075821731434703283262960919232893748488239573476490, 14043460796366809825625631340544734154072024783602894653354532313972836442650, 3845504624369590154153917308407546995991894702250455924670884769418972699294)
pairing1 = (19